In [1]:
hello(world).
hello(prolog).

% Asserting clauses for user:hello/1


In [ ]:
?- hello(X).
?- retry.
?- retry.

X = world

% Retrying goal: hello(X)


X = prolog

% Retrying goal: hello(X)


false

In [3]:
prolog :- forall(
    (Goal = hello(X), call(Goal)),
    (write(Goal), nl)
).

In [4]:
?- prolog.

hello(world)
hello(prolog)

true